# __Models (PyTorch)__

### Text

В этом разделе рассматривается создание и использование модели. Мы будем использовать класс AutoModel, который удобен, когда вы хотите создать экземпляр любой модели из контрольной точки.

Класс AutoModel и все его родственники на самом деле являются простыми оболочками для широкого спектра моделей, доступных в библиотеке. Это умная оболочка, поскольку она может автоматически угадывать подходящую архитектуру модели для вашей контрольной точки, а затем создавать экземпляр модели с этой архитектурой.  

Однако, если вы знаете тип модели, которую хотите использовать, вы можете использовать класс, который определяет ее архитектуру напрямую. Давайте посмотрим, как это работает с моделью BERT.

### `Code`

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

## Creating a Transformer

### `Code`

Первое, что нам нужно сделать для инициализации модели BERT, — это загрузить объект конфигурации:

In [2]:
from transformers import BertConfig, BertModel

config = BertConfig() # Building the config
model = BertModel(config) # Building the model from the config
# Model is randomly initialized!

In [3]:
print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.45.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



## Различные методы загрузки

Модель, которая загружена ранее  `model = BertModel(config)` можно использовать в этом состоянии, но ее веса инициализированы случайными величинами. Ее нужно обучить. Загрузка обученной модели делается с с помощью метода `from_pretrained()`:

### `Code`

In [4]:
model = BertModel.from_pretrained("bert-base-cased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

c:\Users\shaps\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shaps\.cache\huggingface\hub\models--bert-base-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

### Text

Мы не использовали `BertConfig`, а вместо этого загрузили предварительно обученную модель через идентификатор `bert-base-cased`. Это контрольная точка модели, обученная самими авторами BERT.

Теперь эта модель инициализирована со всеми весами контрольной точки. Ее можно использовать напрямую для вывода по задачам, на которых она была обучена, а также ее можно настроить на новую задачу. Обучаясь с предварительно обученными весами, а не с нуля, мы можем быстро добиться хороших результатов.

Веса были загружены и кэшированы (чтобы будущие вызовы метода from_pretrained() не загружали их повторно) в папке кэша, которая по умолчанию находится в ~/.cache/huggingface/transformers. Вы можете настроить папку кэша, установив переменную среды HF_HOME.

Идентификатор, используемый для загрузки модели, может быть идентификатором любой модели в Model Hub, если она совместима с архитектурой BERT. Полный список доступных контрольных точек BERT можно найти здесь:
https://huggingface.co/models?other=bert&sort=trending

## Сохранение модели


Сохранение модели так же просто, как и ее загрузка — мы используем метод `save_pretrained()`, который аналогичен методу from_pretrained():

#### Сохранение в Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
file_path = "/content/drive/My Drive/Hugging_face/bert-base-cased_model_saved"

In [8]:
model.save_pretrained(file_path)

In [13]:
ls '/content/drive/My Drive/Hugging_face/bert-base-cased_model_saved'

config.json  model.safetensors


#### Сохранение на локальном компьютере

In [5]:
model.save_pretrained("bert-base-cased_model_saved")

In [ ]:
ls "bert-base-cased_model_saved"

- `config.json` - атрибуты, необходимые для построения архитектуры модели. Этот файл также содержит некоторые метаданные, например, откуда возникла контрольная точка и какую версию 🤗 Transformers вы использовали, когда последний раз сохраняли контрольную точку.
- `model.safetensors` - словарь состояний: он содержит все веса вашей модели.   Эти два файла идут рука об руку: конфигурация необходима для знания архитектуры вашей модели, в то время как веса модели являются параметрами вашей модели.

## Загрузка модели с диска

In [ ]:
from transformers import AutoModel
model_loaded = AutoModel.from_pretrained("bert-base-cased_model_saved")

## Трансформация исходных данных в тензоры для подачи в модели Transformer

In [ ]:
# Допустим, у нас есть несколько последовательностей:
sequences = ["Hello!", "Cool.", "Nice!"]

Токенизация – это процесс разбиения текста на отдельные части (токены) и преобразование их в индексы из словаря предобученной модели.индексы словаря - `input_ids`! Это список закодированных последовательностей: список списков.   

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
encoded_sequences = tokenizer(sequences, add_special_tokens=True)
encoded_sequences

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

{'input_ids': [[101, 8667, 106, 102], [101, 13297, 119, 102], [101, 8835, 106, 102]], 'token_type_ids': [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]}

Это токены, но модели принимают тензоры, причем только прямоугольные формы. У нас токены имеют одинаковую длину (прямоугольные), поэтому можно сразу их преобразовать в тензор:

In [15]:
import torch
model_inputs = torch.tensor(encoded_sequences["input_ids"])
model_inputs

tensor([[  101,  8667,   106,   102],
        [  101, 13297,   119,   102],
        [  101,  8835,   106,   102]])

In [16]:
# Чтобы подать на вход модели тензоры, надо просто вызывать модель с входными данными:
output = model(model_inputs)

In [17]:
output.last_hidden_state.shape

torch.Size([3, 4, 768])

Хотя модель принимает множество различных аргументов, необходимы только идентификаторы входов. Мы объясним, что делают другие аргументы и когда они требуются позже, но сначала нам нужно поближе рассмотреть токенизаторы, которые создают входы, которые может понять модель Transformer.